# Pinecone Canopy library quick start notebook

**Canopy** is a Software Development Kit (SDK) for AI applications. Canopy allows you to test, build and package Retrieval Augmented Applications with Pinecone Vector Database.

This notebook introduces the quick start steps for working with Canopy library. You can find more details about this project and advanced use in the project [documentation](../README.md).


## Prerequisites

install canopy library

In [ ]:
!pip install -qU \
  datasets \
  cohere \
  canopy-sdk

By default, Canopy uses Pinecone and OpenAI so we need to configure the related API keys.

To get Pinecone free trial API key and environment register or log into your Pinecone account in the [console](https://app.pinecone.io/). You can access your API key from the "API Keys" section in the sidebar of your dashboard, and find the environment name next to it.

You can find your free trial OpenAI API key [here](https://platform.openai.com/account/api-keys). You might need to login or register to OpenAI services.



In [102]:
import os

os.environ["CO_API_KEY"] = "CO_API_KEY"  # dashboard.cohere.com
os.environ["PINECONE_API_KEY"] = 'PINECONE_API_KEY'
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

## Pinecone Documentation Dataset

Now we'll load a crawl from 25/10/23 of pinecone docs [website](https://docs.pinecone.io/docs/).

We will use this data to demonstrate how to build a RAG pipeline to answer questions about Pinecone DB.

In [2]:
from datasets import load_dataset

dataset = load_dataset("jamescalam/ai-arxiv-llama2-ja", split="train")
dataset

Dataset({
    features: ['id', 'text', 'source', 'metadata'],
    num_rows: 318
})

In [3]:
dataset[0]

{'id': '2307.09288-0',
 'text': 'ラマ2：オープンファウンデーションとファインチューンドチャットモデル\nHugo Touvron、Louis Martiny、Kevin Stoney、Peter Albert、Amjad Almahairi、Yasmine Babaei、Nikolay Bashlykov、Soumya Batra、Prajjwal Bhargava、Shruti Bhosale、Dan Bikel、Lukas Blecher、Cristian Canton Ferrer Moya、Chen Guillem、Cucurull David、Esiobu Jude、Fernandes Jeremy、Fu Wenyin、Fu Brian Fuller、Cynthia Gao、Vedanuj Goswami、Naman Goyal、Anthony Hartshorn、Saghar Hosseini、Rui Hou、Hakan Inan、Marcin Kardas、Viktor Kerkez、Madian Khabsa、Isabel Kloumann、Artem Korenev、Punit Singh Koura、Marie-Anne Lachaux、Thibaut Lavril、Jenya Lee、Diana Liskovich、Yinghai Lu、Yuning Mao、Xavier Martinet、Todor Mihaylov、Pushkar Mishra、Igor Molybog、Yixin Nie、Andrew Poulton、Jeremy Reizenstein、Rashi Rungta、Kalyan Saladi、Alan Schelten、Ruan Silva、Eric Michael Smith、Ranjan Subramanian、Xiaoqing Ellen Tan、Binh Tang',
 'source': 'http://arxiv.org/pdf/2307.09288',
 'metadata': {'title': 'Llama 2: Open Foundation and Fine-Tuned Chat Models',
  'en': 'Llama 2: Open Foundation a

We reformat to a list of Canopy `Document` objects:

In [4]:
from canopy.models.data_models import Document

docs = [Document(
    id=x["id"],
    text=x["metadata"]["en"],
    source=x["source"],
    metadata={
        "title": x["metadata"]["title"],
        "ja": x["text"]
    }
) for x in dataset]
len(docs)

318

In [5]:
docs[0]

Document(id='2307.09288-0', text='Llama 2: Open Foundation and Fine-Tuned Chat Models\nL/l.sc/a.sc/m.sc/a.sc /two.taboldstyle : Open Foundation and Fine-Tuned Chat Models\nHugo Touvron\x03Louis MartinyKevin Stoney\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou\nHakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem Korenev\nPunit Singh Koura Marie-Anne Lachaux Thibaut Lavril Jenya Lee Diana Liskovich\nYinghai Lu Yuning Mao Xavier Martinet Todor Mihaylov Pushkar Mishra\nIgor Molybog Yixin Nie Andrew Poulton Jeremy Reizenstein Rashi Rungta Kalyan Saladi\nAlan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan Binh Tang', source='http://arxiv.org/pdf/2307.09288', met

## Creating a Multilingual RecordEncoder

In [6]:
from canopy.knowledge_base.record_encoder.cohere import CohereRecordEncoder

record_encoder = CohereRecordEncoder(model_name="embed-english-v3.0")  # embed-multilingual-v3.0

## Creating a KnowledgBase to store our data for search

The `KnowledgeBase` object is responsible for storing and indexing textual documents.

Once documents are indexed, the `KnowledgeBase` can be queried with a new unseen text passage, for which the most relevant document chunks are retrieved.

The `KnowledgeBase` holds a connection to a Pinecone index and provides a simple API to insert, delete and search textual documents.

The `KnowledgeBase`'s `upsert()` operation is used to index new documents, or update already stored documents. The `upsert` process splits each document's text into smaller chunks, transforms these chunks to vector embeddings, then upserts those vectors to the underlying Pinecone index. At Query time, the `KnowledgeBase` transforms the textual query text to a vector in a similar manner, then queries the underlying Pinecone index to retrieve the top-k most closely matched document chunks.

Here we create a `KnowledgeBase` with our desired index name:

In [86]:
#from canopy.knowledge_base.chunker import MarkdownChunker
# may need when using for japanese text
#chunker = MarkdownChunker(chunk_size=5000)

In [8]:
from canopy.tokenizer import Tokenizer
Tokenizer.initialize()

In [9]:
from canopy.knowledge_base import KnowledgeBase

INDEX_NAME = "ai-arxiv-llama2-en2ja"

kb = KnowledgeBase(  # vector DB
    pinecone_client=pc,
    index_name=INDEX_NAME,
    record_encoder=record_encoder,
#    chunker=chunker
)

In [20]:
# ada-002: 8192 (1536)
# embed-v3: 512  (1024) 768

In [ ]:
# 200-500

In the first one-time setup of a new Canopy service, an underlying Pinecone index needs to be created. If you have created a Canopy-enabled Pinecone index before - you can skip this step.

First, we setup our index specification, this allows us to define the cloud provider and region where we want to deploy our index. You can find a list of all [available providers and regions here](https://docs.pinecone.io/docs/projects).

In [ ]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

Note: Since Canopy uses a dedicated data schema, it is not recommended to use a pre-existing Pinecone index that wasn't created by Canopy's `create_canopy_index()` method.

In [10]:
from canopy.knowledge_base import list_canopy_indexes

if not any(name.endswith(INDEX_NAME) for name in list_canopy_indexes()):
    kb.create_canopy_index(
        spec=spec,
        indexed_fields=["title"]
    )

You can see the index created in Pinecone's [console](https://app.pinecone.io/)

next time we would like to init a knowledge base instance to this index, we can simply call the connect method:

In [11]:
#kb = KnowledgeBase(index_name=INDEX_NAME)
#kb.connect()

> 💡 Note: a knowledge base must be connected to an index before executing any operation. You should call `kb.connect()` to connect  an existing index or call `kb.create_canopy_index(INDEX_NANE)` before calling any other method of the KB

## Upsert data to our KnowledgBase

Each document object can hold id, text, source and metadata:

Now we are ready to upsert our data, with only a single command:

# KB

Text -> tokenizes -> chunked -> embedding model (ada-002 or embedv3) -> upsert

In [12]:
from tqdm.auto import tqdm

batch_size = 20

for i in tqdm(range(0, len(docs), batch_size)):
    kb.upsert(docs[i: i+batch_size])

  0%|          | 0/16 [00:00<?, ?it/s]

Internally, the KnowledgeBase handles all the processing needed to Index the documents. Each document's text is chunked to smaller pieces and encoded to vector embeddings that can be then upserted directly to Pinecone. Later in this notebook we'll learn how to tune and customize this process.

## Query the KnowledgeBase

Now we can query the knowledge base. The KnowledgeBase will use its default parameters like `top_k` to execute the query:

In [13]:
def print_query_results(results):
    for query_results in results:
        print('query: ' + query_results.query + '\n')
        for document in query_results.documents:
            print("japanese: " + document.metadata["ja"])
            print("title: " + document.metadata["title"])
            print("english: " + document.text)
            print('source: ' + document.source)
            print(f"score: {document.score}\n")
            print('-------------------------')

In [14]:
from canopy.models.data_models import Query
results = kb.query([Query(text="what is llama 2?")])

print_query_results(results)

query: what is llama 2?

japanese: Llama 2: オープンファンデーションとファインチューンされたチャットモデル
アラン・シェルテン、ルアン・シルバ、エリック・マイケル・スミス、ランジャン・スブラマニアン、シャオチン・エレン・タン、ビン・タン、ロス・テイラー、アディナ・ウィリアムズ、ジャン・シアン、クアン・プシン・シュウ、イリヤン・ザロフ、ユーチェン・チャン、アンジェラ・ファン、メラニー・カンバドゥール、シャラン・ナラン、オレリアン・ロドリゲス、ロバート・ストイニック、セルゲイ・エドゥノフ、トーマス・シャロム
GenAI、Meta
要約
本研究では、7億から700億のパラメータを持つ事前学習およびファインチューニングされた大規模言語モデル（LLM）のコレクションであるLlama 2を開発およびリリースします。弊社のファインチューニングされたLLMであるL/l.sc/a.sc/m.sc/a.sc /two.taboldstyle-C/h.sc/a.sc/t.scは、対話の使用事例に最適化されています。弊社のモデルは、多くのベンチマークテストでオープンソースのチャットモデルを上回り、私たちの有用性と安全性に関する人間の評価に基づいて、クローズドソースのモデルの代替として適しているかもしれません。ファインチューニングおよび安全性に対するアプローチの詳細な説明を提供します。
title: Llama 2: Open Foundation and Fine-Tuned Chat Models
english: Llama 2: Open Foundation and Fine-Tuned Chat Models
Alan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan Binh Tang
Ross Taylor Adina Williams Jian Xiang Kuan Puxin Xu Zheng Yan Iliyan Zarov Yuchen Zhang
Angela Fan Melanie Kambadur Sharan Narang Aurelien Rodriguez Robert Stojnic
Sergey Eduno

## Query the Context Engine

`ContextEngine` is an object responsible for retrieving the most relevant context for a given query and token budget.  

While `KnowledgeBase` retrieves the full `top-k` structured documents for each query including all the metadata related to them, the context engine in charge of transforming this information to a "prompt ready" context that can later feeded to an LLM. To achieve this the context engine holds a `ContextBuilder` object that takes query results from the knowledge base and returns a `Context` object. The `ContextEngine`'s default behavior is to use a `StuffingContextBuilder`, which simply stacks retrieved document chunks in a JSON-like manner, hard limiting by the number of chunks that fit the `max_context_tokens` budget. More complex behaviors can be achieved by providing a custom `ContextBuilder` class.

In [15]:
from canopy.context_engine import ContextEngine
context_engine = ContextEngine(kb)

In [16]:
import json

result = context_engine.query([Query(text="what is llama 2?", top_k=5)], max_context_tokens=512)

print(result.to_text(indent=2))
print(f"\n# tokens in context returned: {result.num_tokens}")

[
  {
    "query": "what is llama 2?",
    "snippets": [
      {
        "source": "http://arxiv.org/pdf/2307.09288",
        "text": "Llama 2: Open Foundation and Fine-Tuned Chat Models\nAlan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan Binh Tang\nRoss Taylor Adina Williams Jian Xiang Kuan Puxin Xu Zheng Yan Iliyan Zarov Yuchen Zhang\nAngela Fan Melanie Kambadur Sharan Narang Aurelien Rodriguez Robert Stojnic\nSergey Edunov Thomas Scialom\u0003\nGenAI, Meta\nAbstract\nIn this work, we develop and release Llama 2, a collection of pretrained and \ufb01ne-tuned\nlarge language models (LLMs) ranging in scale from 7 billion to 70 billion parameters.\nOur \ufb01ne-tuned LLMs, called L/l.sc/a.sc/m.sc/a.sc /two.taboldstyle-C/h.sc/a.sc/t.sc , are optimized for dialogue use cases. Our\nmodels outperform open-source chat models on most benchmarks we tested, and based on\nourhumanevaluationsforhelpfulnessandsafety,maybeasuitablesubstituteforclosedsource models. We 

As you can see above, although we set `top_k=5`, context engine retreived only 3 results in order to satisfy the 512 tokens limit. Also, the documents in the context contain only the text and source and not all the metadata that is not necessarily needed by the LLM.

## Knowledgeable chat engine

Now we are ready to start chatting with our data!

Canopy's `ChatEngine` is a one-stop-shop RAG-infused Chatbot. The `ChatEngine` wraps an underlying LLM such as OpenAI's ChatGPT, enhancing it by providing relevant context from the user's knowledge base. It also automatically phrases search queries out of the chat history and send them to the knowledge base.

In [17]:
from canopy.chat_engine import ChatEngine
chat_engine = ChatEngine(context_engine)

In [18]:
from typing import Tuple
from canopy.models.data_models import Messages, UserMessage, AssistantMessage

def chat(new_message: str, history: Messages) -> Tuple[str, Messages]:
    messages = history + [UserMessage(content=new_message)]
    response = chat_engine.chat(messages)
    assistant_response = response.choices[0].message.content
    return assistant_response, messages + [AssistantMessage(content=assistant_response)]

In [19]:
from IPython.display import display, Markdown

history = []
response, history = chat("What is llama 2?", history)
display(Markdown(response))

Llama 2 refers to a collection of pretrained and fine-tuned large language models (LLMs) optimized for dialogue use cases. These models range in scale from 7 billion to 70 billion parameters. The Llama 2 models, specifically L/l.sc/a.sc/m.sc/a.sc /two.taboldstyle-C/h.sc/a.sc/t.sc, outperform open-source chat models on most benchmarks and may be considered as a suitable substitute for closed-source models. They are developed with a focus on usability and safety. Llama 2 models have been evaluated and compared to other open-source and closed-source models based on their helpfulness and safety. Source: http://arxiv.org/pdf/2307.09288

In [106]:
response, history = chat("how can I use llama 2?", history)
display(Markdown(response))

To use Llama 2, you can follow these steps:

1. Access the Llama 2 models: You can find the Llama 2 models and resources at the following link: https://ai.meta.com/resources/models-and-libraries/llama/.

2. Review the documentation: It is recommended to review the documentation provided with Llama 2 to understand its capabilities, usage guidelines, and licensing details.

3. Responsible Use: Llama 2 models come with guidelines for responsible development and deployment. It is important to adhere to these guidelines to ensure the safe and ethical use of the models.

4. Fine-tuning and adaptation: Llama 2 provides pretrained models that can be fine-tuned and adapted for various natural language generation tasks. You can refer to the documentation for instructions on how to fine-tune and customize the models for specific use cases.

5. Provide feedback: If you have any feedback or comments on the Llama 2 models, you can follow the instructions provided in the model README or open an issue in the GitHub repository (https://github.com/facebookresearch/llama/) to contribute to community feedback and improvement of model safety.

It is important to comply with the terms of the provided license and the Acceptable Use Policy, which prohibit any uses that would violate applicable policies, laws, rules, and regulations.
(Source: http://arxiv.org/pdf/2307.09288)

In [ ]:
response, history = chat("what is the difference between this llama and llama's you find in argiculture?", history)
display(Markdown(response))

> 💡 Note: Canopy calls the underlying LLM, providing both the user-provided chat history and a generated `Context` prompt. This might surpass the `ChatEngine`'s configured `max_prompt_tokens`. By default, the `ChatEngine` would truncate the oldest messages in the chat history to avoid exceeding this limit. This behavior in configurable, as explained in the [documentation](https://github.com/pinecone-io/canopy/blob/main/src/canopy/chat_engine/chat_engine.py)